# 05_compute_ISC

In [ ]:
import numpy as np
from os.path import join as pjoin
from os.path import isdir
import os
import matplotlib.pyplot as plt
from matplotlib import cm, colors
import mne_bids
import mne
from mne_bids import write_raw_bids, BIDSPath
from scipy import stats
import re
from scipy import signal
import pandas as pd
from scipy import signal, fftpack

In [ ]:
def load_sub_raw_data(bids_root, subject_idx, run_idx):
    """
    load raw meg data. 
    """
    
    if not isinstance(subject_idx, str):
        raise ValueError('subject_dix must be str')
        
    if not isinstance(run_idx, str):
        raise ValueError('run_idx must be str')
    
    subject_data_folder = pjoin(bids_root, 'sub-{}'.format(subject_idx), 'ses-movie', 'meg')
    fname = 'sub-' + subject_idx + '_ses-movie_task-movie_run-' + run_idx + '_meg.ds'
    raw_data_path = pjoin(subject_data_folder, fname)
    raw_data = mne.io.read_raw_ctf(raw_data_path, preload='True')
    
    print('total channels number is {}'.format(len(raw_data.info['chs'])))
    print('sample frequency is {} Hz'.format(raw_data.info['sfreq']))

    return raw_data

In [4]:
def band_split(raw, band_name):
    """

    """
    band_freq = {'delta': [1, 4],
                 'theta': [4, 8],
                 'alpha': [8, 13],
                 'beta': [13, 30],
                 'gamma': [30, 100]}
    
    raw_band = raw.copy().load_data().filter(l_freq=band_freq[band_name][0], h_freq=band_freq[band_name][1])

    return raw_band

In [7]:
def extract_events(bids_root, sub_idx, run_idx):
    """
    
    """
    sub_path = BIDSPath(subject=sub_idx, run=int(run_idx), task='movie', session='movie', processing='preproc', root=bids_root)
    sub_raw = mne_bids.read_raw_bids(sub_path)
    events_sub = mne.events_from_annotations(sub_raw)
    
    return events_sub

In [ ]:
def compute_band_isc(data_sub1, data_sub2, events_sub1, events_sub2, band_name):
    """
    
    """
    
    # get valid channels
    def get_picks(data):
        ch_name_picks = mne.pick_channels_regexp(data.ch_names, regexp='M[LRZ]...-4503')
        type_picks = mne.pick_types(data.info, meg=True)
        picks= np.intersect1d(ch_name_picks, type_picks)
        return picks
        
    picks_sub1 = get_picks(data_sub1)
    picks_sub2 = get_picks(data_sub2)

    # match channels
    picks = np.intersect1d(sub1_picks, sub2_picks)
    
    # mark bad channels
    bad_idx = []
    if len(picks) != len(picks_sub1):
        bad_picks = np.union1d(np.setdiff1d(picks_sub1, picks_sub2), np.setdiff1d(picks_sub2, picks_sub1))
        for chn in bad_picks:
            bad_idx.append(np.where(picks == chn)[0][0])
    
    # band_split
    band_sub1 = band_split(data_sub1, band_name=band_name)
    band_sub2 = band_split(data_sub2, band_name=band_name)
    band_sub1_data = band_sub1.get_data(picks=picks)
    band_sub2_data = band_sub2.get_data(picks=picks)
    
    envlope_sub1 = np.abs(signal.hilbert(band_sub1_data))
    envlope_sub2 = np.abs(signal.hilbert(band_sub2_data))
    
    if len(bad_idx) != 0:
        for idx in bad_idx:
            envlope_sub1[idx,:] = 0
            envlope_sub2[idx,:] = 0
    
    # downsampling
    sample_sub1 = events_sub1[0][1:-1:25,0]
    sample_sub2 = events_sub2[0][1:-1:25,0]
    band_dsamp_sub1 = envlope_sub1.take(sample_sub1, axis=1)
    band_dsamp_sub2 = envlope_sub2.take(sample_sub2, axis=1)
    
    ch_r = []
    ch_p = []
    for ch_idx in np.arange(band_dsamp_sub1.shape[0]):
        r, p = stats.pearsonr(band_dsamp_sub1[ch_idx, :], band_dsamp_sub2[ch_idx, :])
        ch_r.append(r)
        ch_p.append(p)
    
    del data_sub1, data_sub2
    
    return ch_r, ch_p

## define variables

In [ ]:
bids_root = '/nfs/e5/studyforrest/forrest_movie_meg/preproc_data'
sub_list = np.arange(1,12)
run_list = np.arange(1,9)
save_pth = '/nfs/s2/userhome/daiyuxuan/workingdir/MEG-paper/output_data'
band_names = ['delta', 'theta', 'alpha', 'beta', 'gamma']

## compute pre-post preproc ISC

In [ ]:
# pre pre-proc
pre_corr_band_split = {}
for band_name in band_names:
    
    pre_corr_band_split[band_name] = np.zeros((len(run_list), len(sub_list), len(sub_list), 272))
            
    for run in run_list:
        if run >= 7:
            sub_ls = np.arange(2,12)
        else:
            sub_ls = sub_list
            
        for sub1 in sub_ls:
            if sub1 < 10:
                sub1_idx = '0' + str(sub1)
            else:
                sub1_idx = str(sub1)
                
            run_idx = '0'+str(run)
            events_sub1 = extract_events(sub1_idx, run_idx)
            raw_sub1 = load_sub_raw_data(subject_idx=sub1_idx, run_idx=run_idx)

            for sub2 in sub_list:
                if sub2 < 10:
                        sub2_idx = '0' + str(sub2)
                else:
                        sub2_idx = str(sub2)
                
                events_sub2 = extract_events(sub2_idx, run_idx)
                raw_sub2 = load_sub_raw_data(subject_idx=sub2_idx, run_idx=run_idx)
                ch_r, ch_p = compute_band_isc(raw_sub1, raw_sub2, events_sub1, events_sub2, band_name)
                pre_corr_band_split[band_name][run-1][sub1-1][sub2-1] = pre_corr_band_split[band_name][run-1][sub2-1][sub1-1] = np.array(ch_r)
np.save(pjoin(save_pth, 'pre_corr_band_split'), pre_corr_band_split)

# post pre-proc
corr_band_split = {}
for band_name in band_names:
    
    corr_band_split[band_name] = np.zeros((len(run_list), len(sub_list), len(sub_list), 272))
    
    for run in run_list:
        if run >= 7:
            sub_ls = np.arange(2, 12)
        else:
            sub_ls = sub_list
        
        for sub1 in sub_ls:
            if sub1 < 10:
                sub1_idx = '0' + str(sub1)
            else:
                sub1_idx = str(sub1)
            
            sub1_path = BIDSPath(subject=sub1_idx, run=int(run), task='movie', session='movie', processing='preproc', root=bids_root)
            raw_sub1 = mne_bids.read_raw_bids(sub1_path)
            events_sub1 = mne.events_from_annotations(raw_sub1)
            
            for sub2 in np.arange(sub1+1, 12):
                
                if sub2 < 10:
                    sub2_idx = '0' + str(sub2)
                else:
                    sub2_idx = str(sub2)
                
                sub2_path = BIDSPath(subject=sub2_idx, run=int(run), task='movie', session='movie', processing='preproc', root=bids_root)
                raw_sub2 = mne_bids.read_raw_bids(sub2_path)
                events_sub2 = mne.events_from_annotations(raw_sub2)
                ch_r, ch_p = compute_band_isc(raw_sub1, raw_sub2, events_sub1, events_sub2, band_name)
                corr_band_split[band_name][run-1][sub1-1][sub2-1] = corr_band_split[band_name][run-1][sub2-1][sub1-1] = np.array(ch_r)
np.save(pjoin(save_pth, 'corr_band_split'), corr_band_split)
